In [1]:
import pandas as pd
import numpy as np
import pprint
import math
import sys
sys.path.insert(1, '/Users/Hector_Martin/Documents/Labs/music_recommender_project/config.py')
from config import *
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
#Let's initialize Spotipy with our credentials:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret_id))

### Creating the function to collect all the audio features based in a list of songs. This function will also store all the results on a DataFrame:

In [3]:
#Let's get the list of songs of the Hot Songs dataframe:

#Let's read it first:
hotsongs = pd.read_csv('/Users/Hector_Martin/Documents/Labs/music_recommender_project/data/hot100.csv')

#hotsongs.head()
#Getting the list:
hot_songs_list = [song for song in hotsongs['songs']]
#for song in hotsongs['songs']:
#    hot_songs_list.append(song)
 
#hot_songs_list

In [7]:
#Let's do the same with the Not Hot Songs one:

#Let's read it first:
nothotsongs = pd.read_csv('/Users/Hector_Martin/Documents/Labs/music_recommender_project/data/nothotsongs.csv')
#nothotsongs.head()
#Getting the list:
not_hot_songs_list = [song for song in nothotsongs['songs']]
#for song in nothotsongs['track_name']:
#    not_hot_songs_list.append(song)
#not_hot_songs_list

In [5]:
def get_audio_features(list_of_songs):
    
    import math
    import pickle
    import pandas as pd
    import sys
    import spotipy
    from spotipy.oauth2 import SpotifyClientCredentials
    from time import sleep
    import glob
    
    song_ids = []

    black = {'danceability': "Null", 
           'energy': "Null", 
           'key': "Null",
           'loudness': "Null", 
           'mode':"Null", 
           'speechiness': "Null", 
           'acousticness': "Null", 
           'instrumentalness': "Null", 
            'liveness': "Null", 
            'valence': "Null", 
            'tempo': "Null", 
            'type': "Null", 
            'id': "Null", 
            'uri': "Null", 
            'track_href': "Null", 
            'analysis_url': "Null", 
            'duration_ms': "Null", 
            'time_signature': "Null"}
    
    keys = list(black.keys())
    
    df = pd.DataFrame()
    chunks = math.ceil(len(list_of_songs)/1000)
    for i in range(chunks): # chunks = 4 -> 0,1,2,3
        song_ids = []
        if ( i < chunks-1 ):
            j = i + 1
        else:
            j = len(list_of_songs)
        for index, song in enumerate(list_of_songs[1000*i:1000*j]): #[0:1000],[1000:2000],[2000:3000],[3000:]
            print("Looking for song: ",index)
            try:
                songs = sp.search(q=song, limit=1)
                song_ids.append(songs['tracks']['items'][0]['id'])
            except: # Si no esta no se para.
                print(song)
                song_ids.append("")
    
        print("Getting audio features")
        audio_feats = [sp.audio_features(song_id)[0] if ((song_id != None) and ( song_id != "")) else black for song_id in song_ids]
        name = "audio_feats_" + str(i) + ".pkl"
        with open(name, "wb") as handle:
            pickle.dump(audio_feats,handle)
        print("Created file: ",name)
        sleep(30)
        print('Sleeping for 30 seconds')
        
        #Grouping the files together in a list:
        
    pkls = glob.glob("*.pkl")
    pkls.sort()
    pkls
        
    df = pd.DataFrame()
        
    for i, pkl in enumerate(pkls):    
        try:
            print(pkl)
            with open(pkl, "rb") as handle:
                audio_feats = pickle.load(handle)
                audio_feats_df = pd.DataFrame(audio_feats)
                df = pd.concat([df,audio_feats_df],axis = 0).reset_index(drop=True)
                    
        except:
            print("Corrupted",pkl)
            continue
                             
    return df

In [ ]:
get_audio_features(not_hot_songs_list)

Looking for song:  0


In [1]:
### Function to concatenate the DataFrames:

In [11]:
def add_audio_features(df, audio_features_df, csvname):  
    if df.shape <=(100, df.columns):
        df_concat = pd.concat([df, audio_features_df], axis =1)
        df_concat = df_concat[df_concat['mode'] != 'Null']
        df_concat = df_concat.reset_index(drop=True)
        df_concat.to_csv(csvname+'.csv', index=False)
        return df_concat
    else:
        df = df.drop(df.index[1000:2000]).reset_index(drop=True)
        df_concat = pd.concat([df, audio_features_df], axis =1)
        df_concat = df_concat[df_concat['mode'] != 'Null']
        df_concat = df_concat.reset_index(drop=True)
        df_concat.to_csv(csvname+'.csv', index=False)
        return df_concat

In [30]:
def concatallsongs(dfhotconcat,dfnothotconcat,csvname):
    df_allsongsconcat = pd.concat([dfhotconcat, dfnothotconcat], axis =0).reset_index(drop=True)
    df_allsongsconcat = df_allsongsconcat[df_allsongsconcat['mode'] != 'Null']
    df_allsongsconcat = df_allsongsconcat.reset_index(drop=True)
    df_allsongsconcat.to_csv(csvname+'.csv', index=False)
    return df_allsongsconcat